In [1]:
!pip install mlxtend
!pip install refractio[snowflake]

     |████████████████████████████████| 1.5MB 4.2MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 11.6MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 86.5MB/s eta 0:00:01
     |████████████████████████████████| 307kB 75.7MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 76.7MB/s eta 0:00:01
     |████████████████████████████████| 11.1MB 84.6MB/s eta 0:00:01
     |████████████████████████████████| 9.2MB 81.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 82.4MB/s eta 0:00:01
     |████████████████████████████████| 348kB 77.2MB/s eta 0:00:01
     |████████████████████████████████| 512kB 73.5MB/s eta 0:00:01
     |████████████████████████████████| 4.5MB 72.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 17.5MB/s eta 0:00:01
     |████████████████████████████████| 112kB 83.0MB/s eta 0:00:01
     |████████████████████████████████| 4.7MB 74.2MB/s eta 0:00:01
     |████████████████████████████████| 307kB 78.7MB/s eta 0

     |████████████████████████████████| 4.4MB 78.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 23.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 21.9MB/s eta 0:00:01
     |████████████████████████████████| 163kB 83.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 84.6MB/s eta 0:00:01
     |████████████████████████████████| 450kB 78.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/08/aa/cc0199a5f0ad350994d660967a8efb233fe0416e4639146c089643407ce6/packaging-24.1-py3-none-any.whl
     |████████████████████████████████| 153kB 74.7MB/s eta 0:00:01
     |████████████████████████████████| 143kB 81.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 16.5MB/s eta 0:00:01
     |████████████████████████████████| 40.0MB 77.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py3-none-any.whl
     |██████████████████

In [2]:
from refractio import snowflake
snowflake.get_connection()
dataset = snowflake.execute_query(f" SELECT * FROM RETAIL_DB.RETAIL_MKT_BSKT_SCHEMA.product_sales")
dataset

User name picked from OS env: vikaskumar.10710602@ltimindtree.com
Fetching connections created by vikaskumar.10710602@ltimindtree.com user
Connection names fetched ['RETAIL_MKT_BSKT_CXN'], created by vikaskumar.10710602@ltimindtree.com
Exception occurred in getting snowflake connection: 'connectionSources'
User name picked from OS env: vikaskumar.10710602@ltimindtree.com
Fetching connections created by vikaskumar.10710602@ltimindtree.com user
Connection names fetched ['RETAIL_MKT_BSKT_CXN'], created by vikaskumar.10710602@ltimindtree.com
Exception occurred in getting snowflake connection: 'connectionSources'
Exception occurred in execute_query: 'NoneType' object has no attribute 'cursor'


In [16]:
import pandas as pd
product_set=dataset.groupby("ORDER_ID")["PRODUCT"].apply(set).to_dict()

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
filter_basket = {key:value for key, value in product_set.items() if len(value)>1}
List_order_id=list(filter_basket.keys())
List_product_id=[list(values) for values in filter_basket.values()]
dict1={"ORDER_ID":List_order_id,"PRODUCT":List_product_id}
df1=pd.DataFrame(dict1)
print(df1)

           ORDER_ID                                            PRODUCT
0      AE-2013-1130  [green tea, frozen vegetables, chocolate, eggs...
1      AE-2013-1530  [chocolate, turkey, mushroom cream sauce, cook...
2      AE-2014-2840  [yogurt, newspapers, sugar, baking powder, fru...
3      AE-2014-4120                 [other vegetables, red/blush wine]
4      AG-2013-2840  [green tea, french fries, grated cheese, avocado]
...             ...                                                ...
12160  ZI-2014-6740  [tropical fruit, yogurt, newspapers, brown bre...
12161  ZI-2014-6750                        [salad, french fries, eggs]
12162  ZI-2014-7160      [meat, whole milk, chicken, other vegetables]
12163  ZI-2014-9540  [curd, yogurt, whole milk, shopping bags, cann...
12164  ZI-2014-9550                            [sugar, UHT-milk, soda]

[12165 rows x 2 columns]


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Create a list of unique items
unique_items = sorted(set(item for sublist in df1['PRODUCT'] for item in sublist))
 
# Initialize an empty dictionary to hold binary values for each item
binary_data = {item: [] for item in unique_items}
 
# Iterate over each row in the original DataFrame
for bought_together in df1['PRODUCT']:
    # Initialize a binary row for the current transaction
    binary_row = {item: 0 for item in unique_items}
    # Set the value to 1 if the item was bought in the transaction
    for item in bought_together:
        binary_row[item] = 1
    # Append the binary row to the binary_data dictionary
    for item, value in binary_row.items():
        binary_data[item].append(value)
 
#Create a DataFrame from the dictionary
binary_df = pd.DataFrame(binary_data)
 
# Concatenate with the original DataFrame
binary_df2 = pd.concat([df1['ORDER_ID'], binary_df], axis=1)
 
# Display the binary DataFrame
print(binary_df)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


       Instant food products  UHT-milk  abrasive cleaner  almonds  \
0                          0         0                 0        0   
1                          0         0                 0        0   
2                          0         0                 0        0   
3                          0         0                 0        0   
4                          0         0                 0        0   
...                      ...       ...               ...      ...   
12160                      0         0                 0        0   
12161                      0         0                 0        0   
12162                      0         0                 0        0   
12163                      0         0                 0        0   
12164                      0         1                 0        0   

       antioxydant juice  artif. sweetener  asparagus  avocado  babies food  \
0                      0                 0          0        0            0   
1            

In [19]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
# Load your dataset
frequent_itemsets = apriori(binary_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
# Filter rules based on desired metrics
#rules
filtered_rules = pd.DataFrame(rules[(rules['confidence'] > 0.01) & (rules['lift'] > 0.05)])
#
# # Print the filtered rules
print(filtered_rules)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/tmp/pip_packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


                      antecedents                   consequents  \
0                          (beef)            (other vegetables)   
1              (other vegetables)                        (beef)   
2                          (beef)                  (rolls/buns)   
3                    (rolls/buns)                        (beef)   
4               (root vegetables)                        (beef)   
..                            ...                           ...   
629      (yogurt, tropical fruit)                  (whole milk)   
630  (tropical fruit, whole milk)                      (yogurt)   
631                      (yogurt)  (tropical fruit, whole milk)   
632                  (whole milk)      (yogurt, tropical fruit)   
633              (tropical fruit)          (yogurt, whole milk)   

     antecedent support  consequent support   support  confidence      lift  \
0              0.040855            0.150760  0.016441    0.402414  2.669232   
1              0.150760            0.

In [20]:
frequent_itemsets

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.026223,(UHT-milk)
1,0.012495,(almonds)
2,0.020140,(avocado)
3,0.013975,(baking powder)
4,0.040855,(beef)
...,...,...
398,0.011097,"(root vegetables, whole milk, rolls/buns)"
399,0.013646,"(yogurt, whole milk, rolls/buns)"
400,0.010111,"(root vegetables, whole milk, tropical fruit)"
401,0.012495,"(root vegetables, yogurt, whole milk)"


In [21]:
rules

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(beef),(other vegetables),0.040855,0.150760,0.016441,0.402414,2.669232,0.010281,1.421118,0.651998
1,(other vegetables),(beef),0.150760,0.040855,0.016441,0.109051,2.669232,0.010281,1.076544,0.736377
2,(beef),(rolls/buns),0.040855,0.140567,0.011508,0.281690,2.003954,0.005766,1.196465,0.522326
3,(rolls/buns),(beef),0.140567,0.040855,0.011508,0.081871,2.003954,0.005766,1.044674,0.582927
4,(root vegetables),(beef),0.086067,0.040855,0.014386,0.167144,4.091166,0.010869,1.151634,0.826724
...,...,...,...,...,...,...,...,...,...,...
629,"(yogurt, tropical fruit)",(whole milk),0.024003,0.195890,0.012906,0.537671,2.744763,0.008204,1.739260,0.651303
630,"(tropical fruit, whole milk)",(yogurt),0.035347,0.109248,0.012906,0.365116,3.342091,0.009044,1.403016,0.726465
631,(yogurt),"(tropical fruit, whole milk)",0.109248,0.035347,0.012906,0.118134,3.342091,0.009044,1.093877,0.786735
632,(whole milk),"(yogurt, tropical fruit)",0.195890,0.024003,0.012906,0.065883,2.744763,0.008204,1.044834,0.790526


In [22]:
filtered_rules = filtered_rules.rename(columns={'antecedents': 'product' , 'consequents': 'recommendation' , 'antecedent support': 'product_support','consequent support': 'recommendation_support' })


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
import json
filtered_rules['product'] = list(filtered_rules['product'])
file = 'filtered_rules.csv'
filtered_rules.to_csv(file , index=False)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
snowflake.execute_query(f"PUT file://{file} @RETAIL_DB.RETAIL_MKT_BSKT_SCHEMA.RETAIL_MKT_BSKT_STAGE")

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


User name picked from OS env: vikaskumar.chauhan@fosfor.com
Fetching connections created by vikaskumar.chauhan@fosfor.com user
Connection names fetched ['Snowflake_vikas', 'basket_conn', 'pharma', 'RETAIL_MKT_BSKT_CXN'], created by vikaskumar.chauhan@fosfor.com
Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7f9b25f39d00>
Please close the connection after use!
Exception occurred in execute_query: Unknown error


In [25]:
#pip install fosforml

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
#from fosforml import *

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [26]:
input_file = pd.read_csv("Order_Inventory.csv")
input_file

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Order_Date,Order_ID,Product_ID,Product,Quantity,Unit Price,Ship Mode,Total_Sales,Customer_ID
0,1/31/2021 0:00,BU-2013-7380,10002439,shrimp,167,74.69,Standard Class,12473.23,MH-7455
1,1/31/2021 0:00,BU-2013-7380,10000353,almonds,278,15.28,Standard Class,4247.84,MH-7455
2,1/31/2021 0:00,BU-2013-7380,10000994,avocado,324,46.33,Standard Class,15010.92,MH-7455
3,1/31/2021 0:00,BU-2013-7380,10000475,vegetables mix,124,58.22,Standard Class,7219.28,MH-7455
4,1/31/2021 0:00,BU-2013-7380,10003338,green grapes,680,86.31,Standard Class,58690.80,MH-7455
...,...,...,...,...,...,...,...,...,...
72624,3/30/2023 0:00,US-2013-151225,10004883,chicken,290,86.72,Standard Class,25148.80,GK-14620
72625,3/30/2023 0:00,US-2013-151225,10001990,tropical fruit,314,56.47,Standard Class,17731.58,GK-14620
72626,3/30/2023 0:00,US-2013-151225,10000425,other vegetables,447,92.13,Standard Class,41182.11,GK-14620
72627,3/30/2023 0:00,US-2013-151225,10001410,vinegar,701,89.69,Standard Class,62872.69,GK-14620


In [27]:
req = requests.Request()
req.file = "Order_Inventory.csv"

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
pd.read_csv(req.file)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Order_Date,Order_ID,Product_ID,Product,Quantity,Unit Price,Ship Mode,Total_Sales,Customer_ID
0,1/31/2021 0:00,BU-2013-7380,10002439,shrimp,167,74.69,Standard Class,12473.23,MH-7455
1,1/31/2021 0:00,BU-2013-7380,10000353,almonds,278,15.28,Standard Class,4247.84,MH-7455
2,1/31/2021 0:00,BU-2013-7380,10000994,avocado,324,46.33,Standard Class,15010.92,MH-7455
3,1/31/2021 0:00,BU-2013-7380,10000475,vegetables mix,124,58.22,Standard Class,7219.28,MH-7455
4,1/31/2021 0:00,BU-2013-7380,10003338,green grapes,680,86.31,Standard Class,58690.80,MH-7455
...,...,...,...,...,...,...,...,...,...
72624,3/30/2023 0:00,US-2013-151225,10004883,chicken,290,86.72,Standard Class,25148.80,GK-14620
72625,3/30/2023 0:00,US-2013-151225,10001990,tropical fruit,314,56.47,Standard Class,17731.58,GK-14620
72626,3/30/2023 0:00,US-2013-151225,10000425,other vegetables,447,92.13,Standard Class,41182.11,GK-14620
72627,3/30/2023 0:00,US-2013-151225,10001410,vinegar,701,89.69,Standard Class,62872.69,GK-14620


In [ ]:
from refractml import *
from refractml.constants import MLModelFlavours
import requests

@scoring_func
def score(model, request):
    import json
    payload = request.json["payload"]
    if isinstance(request.json["payload"],str):
        payload_data = eval(payload)
        if isinstance(payload_data['Geography'], str):
            data_json = eval(payload)
            data = pd.DataFrame([data_json])
            prediction = pd.DataFrame(model.predict(data))
            return prediction[0].to_list()[0]
        elif isinstance(payload_data['Geography'], dict):
            data = pd.DataFrame(eval(payload))
            prediction = pd.DataFrame(model.predict(data))
            return prediction[0].tolist()
        elif isinstance(payload_data['Geography'], list):
            data = pd.DataFrame(payload_data)
            prediction = model.predict(data)
            return prediction.tolist()
    return "This method is not allowed"

payload = str(X.iloc[158].to_dict())
req = requests.Request()
req.json= {'payload': payload}
print({'payload': payload})




In [9]:
snowflake.get_connection('RETAIL_MKT_BSKT_CXN')

Existing connection object fetched: <snowflake.connector.connection.SnowflakeConnection object at 0x7f9b967c2b20>
Please close the connection after use!


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
